In [1]:
import sys
import datetime
import time



from sklearn.decomposition import NMF
import numpy as np
import pandas as pd
from datetime import date
today = str(date.today())

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

BelugaMode = False
#saveDHSmat = True


# several parameters
axis_fontsize = 30
myrandom = 0
customthresh = 0.5

def mytime():
	ts = time.time()
	sttime = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d_%H:%M:%S')
	return sttime
print('starting at ', mytime(), flush=True)

def increase_axis_fontsize():
	ax = plt.gca()
	ticklabels = ax.get_xticklabels()
	for label in ticklabels:
		label.set_fontsize(axis_fontsize)
		label.set_family('serif')
	ticklabels = ax.get_yticklabels()
	for label in ticklabels:
		label.set_fontsize(axis_fontsize)
		label.set_family('serif')
		
def make_bar_plot(Nrelevant, BarMatrix, bargraph_out, plotClusterMode=False, barsortorder=[], clusterTopLabels=[], myNMFcomps=16):
	if len(barsortorder)<1:
		barsortorder = np.arange(Nrelevant)
	ttt = np.arange(Nrelevant)
	start = 0
	end = Nrelevant
	ground_pSample = ttt*0
	plt.clf()
	plt.figure(figsize=(150,40))
	plt.bar(ttt[start:end], BarMatrix[0,start:end][barsortorder], color='r',
             bottom=ground_pSample[start:end], alpha=0.75)
	ground_pSample = BarMatrix[0,start:end][barsortorder]
	for i in range(1,myNMFcomps):
		plt.bar(ttt[start:end],BarMatrix[i,start:end][barsortorder], bottom = ground_pSample, color=Comp_colors[i], alpha=0.75)
		ground_pSample = np.sum(BarMatrix[0: i+1,start:end], axis=0)[barsortorder]
	increase_axis_fontsize()
	plt.ylabel('sum of signal in matrix',fontsize=70)
	#plt.title('Full Sample',fontsize=70)
	samplenamesize = 27
	thebottom = 0.25
	if (BelugaMode):
		samplenamesize = 11
		thebottom = 0.15
	if(plotClusterMode):
		plt.xticks(ttt, ttt.astype(str), rotation='vertical', fontsize=samplenamesize)
		if len(clusterTopLabels) > 0:
			ax = plt.gca()
			ax2 = ax.twiny()
			ax2.set_xticks(ttt)
			ax2.set_xticklabels(clusterTopLabels.astype(str), rotation=90, fontsize=samplenamesize)
			#ax.xaxis.tick_top()
			#plt.xticks(ttt, clusterTopLabels.astype(str), rotation='vertical', fontsize=samplenamesize)
	else:
		plt.xticks(ttt, names[barsortorder], rotation='vertical', fontsize=samplenamesize)	
	plot_margin = 5
	plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=thebottom)
	plt.savefig(bargraph_out)
	plt.close()	

#Nsamples
if (BelugaMode):
	Nsamples = 827
	NMFcomps = 0 #will be assigned later
	B = pd.read_table('colTitles_friendly.txt', header=None)
	names = B[1].values
else:
	Nsamples = 176
	NMFcomps = 16 #has to be set here since NMF will be done on the spot
	B = pd.read_table('rowTitles.txt', header=None)
	names = B[0].values


if (BelugaMode):
	if (len(sys.argv) < 3):
		print ('syntax: blah.py pickle1(daslog) pickle2(daslog2)')
		sys.exit()
	finname1 = str(sys.argv[1])
	finname2 = str(sys.argv[2])
	print('starting read at ', mytime(), flush=True)
	A = pd.read_table('masterListPresenceAbsence_827samples_FDR0.0010_hg38.bed')
	print('finished read at ', mytime(), flush=True)
	print(A.head())
	print(A.shape)
	a = A.drop([A.columns[0], A.columns[1], A.columns[2]], axis=1).values
	a = a.T
	print('finished drop at ', mytime(), flush=True)
	del(A)
else:
	IslandSize = 400
	finname = 'mtxlist.txt'
	f = open(finname, 'r')
	dars = f.readlines()
	featurelist = []
	for line in dars:
		g = np.loadtxt(line.strip())
		#daslog = np.log10(g+1)
		daslog = (g > 0).astype(int) 
		if (daslog.shape[1] != IslandSize):
			daslog = daslog[:,:IslandSize]
		featurelist.append(daslog)
	featurelist = np.array(featurelist)
	a = np.copy(featurelist[0])
	for i in range(499):
		a= np.append(a, featurelist[i+1], axis=1)
	#print (a.shape)


if (BelugaMode):
	print('starting read NMF results at ', mytime(), flush=True)
	BasisMat = np.load(finname1)
	MixtureMat = np.load(finname2)
else:
	model = NMF(n_components=NMFcomps, init='random', random_state=0)
	BasisMat = model.fit_transform(a) 
	MixtureMat = model.components_
	print(model.reconstruction_err_)

reconstruction = np.dot(BasisMat, MixtureMat)
NMFcomps = BasisMat.shape[1]
NDHS = MixtureMat.shape[1]

Comp_colors = ['red', 'tan', 'lime','blue','m','k','c', 'coral', 'indigo','darkgreen','orange','grey','gold', 'lightskyblue', 'peru', 'olive']
if (NMFcomps>16):
	from matplotlib import colors as mcolors
	colornames = list(mcolors.CSS4_COLORS.keys())
	count = 16
	while (count < NMFcomps):
		newcolor = colornames[np.random.randint(0,len(colornames))]
		trialcount = 0
		while ((newcolor in Comp_colors) and (trialcount < 100)):
			newcolor = colornames[np.random.randint(0,len(colornames))]
			trialcount+=1
		Comp_colors.append(newcolor)
		count+=1
		
#for now lets just do unweighted version


starting at  20170630_17:03:49
1081.9722798050345


In [2]:


rawnormed_Basisums = np.sum(BasisMat, axis=1)
rawnormed_VecRats = np.copy(BasisMat)
for i, rat in  enumerate(rawnormed_VecRats):
	if (rawnormed_Basisums[i]>0):
		rat/=rawnormed_Basisums[i]

rawnormed_Mixturesums = np.sum(MixtureMat.T, axis=1)
rawnormed_MixtureRats = np.copy(MixtureMat.T)
rawnormed_MixtureRats = rawnormed_MixtureRats[rawnormed_Mixturesums>0]
rawnormed_Mixturesums = rawnormed_Mixturesums[rawnormed_Mixturesums>0]
for i, rat in  enumerate(rawnormed_MixtureRats):
    rat/=rawnormed_Mixturesums[i]

from sklearn.cluster import KMeans

kmeans_normed_sample_orig = KMeans(n_clusters=NMFcomps*2, random_state=0).fit(rawnormed_VecRats)
ClusterMeans = kmeans_normed_sample_orig.cluster_centers_
ClusterPredictions = kmeans_normed_sample_orig.predict(rawnormed_VecRats)

In [9]:
#which things in which cluster

Clusters = []
for i in range(NMFcomps*2):
    current_cut = np.argwhere(ClusterPredictions==i).T[0]
    print(i, len(ClusterPredictions[current_cut]), names[current_cut])
    Clusters.append([i, names[current_cut], current_cut ])


0 6 ['DS11752_HGF_LN2307' 'DS13219_IMR90_LN2741' 'DS13368_HMF_LN2775'
 'DS13480_HCFaa_LN2800' 'DS13513_AoAF_LN2811' 'DS13573_HPdLF_LN2823']
1 5 ['DS23530_h.KBM_7_LN20483' 'DS12543_NB4_LN2556' 'DS25969A_h.CD34+_LN26316'
 'DS31642A_h.CD1_LN28530' 'DS17215_h.CD14+_LN3909']
2 10 ['DS23286_h.f.Retina_LN20434' 'DS23341_h.f.Retina_LN20451'
 'DS23368A_h.f.Retina_LN20458' 'DS23833A_h.f.Eye_LN20656'
 'DS24728A_h.f.Eye_LN21204' 'DS24796A_h.f.Eye_LN21208'
 'DS24833A_fSpinalCord_LN21289' 'DS24872A_fBrain_LN21323'
 'DS23339E_h.f.Retina_LN26813' 'DS23849C_fBrain_LN28707']
3 12 ['DS17885_CD8+_LN10088' 'DS17212_CD4+_LN10089' 'DS14108_h.Tn_LN2964'
 'DS14702_hTreg_LN3156' 'DS17189_CD56+_LN3905' 'DS17198_CD3+_LN3906'
 'DS17203_CD8+_LN3907' 'DS17597_TH2_LN4024' 'DS17603_iTH2_LN4026'
 'DS17881_CD4+_LN4097' 'DS18015_TH1_LN4128' 'DS18018_iTH1_LN4129']
4 10 ['DS24235A_h.SJSA_1_LN20839' 'DS24567A_h.Daoy_LN20970'
 'DS24640A_h.MG_63_LN21068' 'DS12829_NHLF_LN2632' 'DS26313A_h.ELR_LN26395'
 'DS14426_HSMM_LN3058' 'D

In [5]:
#old way of doing things
for i in range(NMFcomps):
    current_col = rawnormed_VecRats[:,i]
    current_cut = current_col > 0.8
    fung = np.argwhere(current_cut).T[0]
    print(Comp_colors[i], len(rawnormed_Basisums[current_cut]), names[current_cut], fung)
    #print(Comp_colors[i], len(rawnormed_Basisums[current_cut]), names[current_cut], fung, file=f)
    #if ( len(borfRats[current_cut]) >= Num_for_cluster):
        #Clusters.append([first16colors[i], names[current_cut], fung ])
    #already_clustered[current_cut] = True


red 1 ['DS23929C_h.brain_LN20744'] [29]
tan 1 ['DS13411_HPAF_LN2784'] [105]
lime 4 ['DS10518_SAEC_LN1926' 'DS22985_MCF10a_ER_SRC_LN20386'
 'DS22991_MCF10a_ER_SRC_LN20387' 'DS22980_MCF10a_ER_SRC_LN20496'] [11 15 16 25]
blue 3 ['DS15153_h.IPS_LN10036' 'DS11909_H7_hESC_LN2365'
 'DS34890A_h.FUCCI.cells_LN30023'] [  3  56 125]
m 3 ['DS13150_HMVEC_dLyNeo_LN2725' 'DS13242_HMVEC_dBlNeo_LN2746'
 'DS13337_HMVEC_dBlAd_LN2768'] [ 96 100 101]
k 3 ['DS17767_fMuscle_back_LN4072' 'DS20239_fMuscle_leg_LN4772'
 'DS20242_fMuscle_trunk_LN4773'] [151 166 167]
c 5 ['DS17885_CD8+_LN10088' 'DS17189_CD56+_LN3905' 'DS17198_CD3+_LN3906'
 'DS17203_CD8+_LN3907' 'DS17603_iTH2_LN4026'] [  7 142 143 144 149]
coral 3 ['DS17313_h.f.Intestine.Lg_LN3928' 'DS17317_fIntestine.Sm_LN3929'
 'DS20770_Small_Bowel_Mucosa_LN4899'] [146 147 174]
indigo 3 ['DS23983C_fLung_LN20738' 'DS24726A_fLung_LN21205' 'DS14724_fLung_LN3167'] [ 28  43 132]
darkgreen 5 ['DS24094A_h.CD34+_LN20866' 'DS24074A_h.CD34+_LN21381'
 'DS25687A_h.CD34+_LN26

In [13]:
import scipy.spatial.distance as spdist
import matplotlib.markers as themarkers 



In [18]:
for i, cluster in enumerate(Clusters):

	print ('doing cluster ',i)
	# mean sample-wise component vector for this cluster
	mean_of_cluster = ClusterMeans[i]
	recon_cluster_DHS = np.dot(mean_of_cluster, MixtureMat)
	cluster_length = len(cluster[1])
	print('cluster length ',cluster_length)
	print('name of samples', cluster[1])
	kar = np.tile(recon_cluster_DHS, cluster_length).reshape(cluster_length, NDHS)
	#print(kar.shape)
	sample_cosdists = spdist.cdist(BasisMat, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()    
	DHScosdists = spdist.cdist(MixtureMat.T, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()
	
	closers = np.argsort(sample_cosdists)
	#threshold for what counts as a positive prediction in reconstruction matrix
	chosenthresh = 0.5
	#threshold for cosine similarity distance for samples to be included in this module. 0.1 gives about the same samples as in the original cluster for 80% component dominance. 
	cosdist_sample_thresh = 0.3

	#threshold for cosine similarity distance for 	
	cosdist_DHS_thresh = 0.3

    
	#print(np.sort(sample_cosdists)[0:Num_samples])
	#print(len(sample_cosdists[sample_cosdists<0.4]))
	Num_samples = len(sample_cosdists[sample_cosdists<cosdist_sample_thresh])
	print(' Num samples', Num_samples)


	DHSappear_cut = np.dot(mean_of_cluster, MixtureMat)>chosenthresh
	DHScloseness = np.argsort(DHScosdists[DHSappear_cut])
	Num_DHSs = len(DHScosdists[DHSappear_cut][DHScosdists[DHSappear_cut] < cosdist_DHS_thresh])
	print(' Num_DHSs', Num_DHSs)
	print('samples meeting threshold ',names[closers[0:Num_samples]])

	sorted_Module = np.dot(BasisMat[closers[0:Num_samples]], MixtureMat.T[DHSappear_cut][DHScloseness][0:Num_DHSs].T)

	corresponding_data = a[closers[0:Num_samples]][:,np.argwhere(DHSappear_cut == True).T[0][DHScloseness][0:Num_DHSs]]
	flatsorted_Module = sorted_Module.flatten()
	flatcorresponding_data = corresponding_data.flatten()
	if (len(flatsorted_Module) > 0):
		density_in_recon = len(flatsorted_Module[flatsorted_Module>chosenthresh])/len(flatsorted_Module)
	else:
		density_in_recon = 0
	if(len(flatcorresponding_data) > 0):
		density_in_data = len(flatcorresponding_data[flatcorresponding_data>chosenthresh])/len(flatcorresponding_data)
	else:
		density_in_data = 0
	TP =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
	FP =len(flatcorresponding_data[ np.invert(flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
	FN =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * np.invert(flatsorted_Module>chosenthresh)])
	if ((TP + FN ) > 0):
		recall = TP / (TP + FN)
	else: 
		recall = 0
	if ((TP +FP) > 0):
		precision = TP / (TP + FP)
	else:
		precision = 0
	print('density in reconstruction,  density in data,  precision,  recall')
	print(density_in_recon, density_in_data, precision, recall  )
	firstcut = np.copy(DHSappear_cut)
	firstcut[DHSappear_cut] *= (DHScosdists[DHSappear_cut] < cosdist_DHS_thresh)
	list1 = [Num_samples, Num_DHSs, chosenthresh, cosdist_sample_thresh,  cosdist_DHS_thresh, density_in_recon, density_in_data, precision, recall]
	theheader =  ' '.join(str(e) for e in list1)
	namestr = ' '.join(names[closers[0:Num_samples]])
	theheader +=' '+namestr
	#foutname = today+'Module'+str(i)+'.txt'
	#np.savetxt(foutname, firstcut, fmt='%i', header=theheader)
	#'''

doing cluster  0
cluster length  6
name of samples ['DS11752_HGF_LN2307' 'DS13219_IMR90_LN2741' 'DS13368_HMF_LN2775'
 'DS13480_HCFaa_LN2800' 'DS13513_AoAF_LN2811' 'DS13573_HPdLF_LN2823']
 Num samples 22
 Num_DHSs 1259
samples meeting threshold  ['DS13368_HMF_LN2775' 'DS11752_HGF_LN2307' 'DS13219_IMR90_LN2741'
 'DS13573_HPdLF_LN2823' 'DS13513_AoAF_LN2811' 'DS12684_HIPEpiC_LN2583'
 'DS13480_HCFaa_LN2800' 'DS34574A_h.CEC_LN29827' 'DS11949_SKMC_LN2373'
 'DS14790_HAsp_LN3184' 'DS21042_h.H1.polyMSC_LN10043' 'DS13411_HPAF_LN2784'
 'DS12447_HCPEpiC_LN2530' 'DS12501_HCF_LN2542' 'DS13981_HVMF_LN2925'
 'DS12663_HAEpiC_LN2579' 'DS12599_HCM_LN2565' 'DS13390_HPF_LN2780'
 'DS14426_HSMM_LN3058' 'DS20548_LHCN_M2_LN4870' 'DS12863_NHDF_Ad_LN2643'
 'DS12374_AG10803_LN2497']
density in reconstruction,  density in data,  precision,  recall
0.9942595133222616 0.915733988013575 0.9190602418388467 0.9978709982652578
doing cluster  1
cluster length  5
name of samples ['DS23530_h.KBM_7_LN20483' 'DS12543_NB4_LN25